# Урок 7
## Линейная регрессия. Однофакторный дисперсионный анализ

### Задача 1

#### Условие: 
Дана матрица объект-признак

```
X = [[ 1.22401313, 2.30868478, 3.03636353, 2.69287214],
[-0.18757272, 1.30337355, 5.12093014, 3.46363202],
[-0.81094525, 1.82463398, 5.79686488, 1.86159445],
[ 0.75129018, 2.67392052, 3.65529809, 1.66746094],
[ 0.00972362, 1.97367255, 2.50594319, 1.69755173],
[-0.62972637, 0.77750764, 2.84124027, 4.54410559],
[ 2.29536229, 1.81206697, 1.95026215, 1.51874636],
[ 0.0920418 , 2.26971361, 7.47708735, 2.61081203],
[ 2.39252799, 3.17563985, 3.61420599, 5.10773362],
[ 0.54983815, 2.87988651, 1.65752765, 1.59635987]]
```

и значения целевой переменной

```
y = [ 9.26193358, 9.700363 , 8.67214805, 8.74796974, 6.18689108,
7.53312713, 7.57643777, 12.44965478, 14.29010746, 6.68361218]
```

1. Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат.

2. Является ли значимым получившееся уравнение регрессии?

#### Решение:

##### Пункт 1:

In [38]:
import numpy as np

In [39]:
X = np.array([[ 1.22401313, 2.30868478, 3.03636353, 2.69287214],
              [-0.18757272, 1.30337355, 5.12093014, 3.46363202],
              [-0.81094525, 1.82463398, 5.79686488, 1.86159445],
              [ 0.75129018, 2.67392052, 3.65529809, 1.66746094],
              [ 0.00972362, 1.97367255, 2.50594319, 1.69755173],
              [-0.62972637, 0.77750764, 2.84124027, 4.54410559],
              [ 2.29536229, 1.81206697, 1.95026215, 1.51874636],
              [ 0.0920418 , 2.26971361, 7.47708735, 2.61081203],
              [ 2.39252799, 3.17563985, 3.61420599, 5.10773362],
              [ 0.54983815, 2.87988651, 1.65752765, 1.59635987]])

Y = np.array([ 9.26193358, 9.700363 , 8.67214805, 8.74796974, 6.18689108,
               7.53312713, 7.57643777, 12.44965478, 14.29010746, 6.68361218])

Для каждой пары признаков построим модель линейной регрессии.

Считать коэффициенты линейной регрессии будем по формуле:

$$ b = (X^TX)^{-1}X^Ty $$

Т.к. в данной формуле $X$ - расширенная матрица, то необходимо добавить столбец из единиц:
```python
X = np.ones((x.shape[0], 1))
```

Найдем предсказанные значения моделью:

$y_{pred} = X * b$

Найдем ошибки модели:

$E = y - y_{pred}$

Найдем коэффициент детерминации:

$$R^2 = 1 - \frac {\sigma^2_E} {\sigma^2_y}$$

In [40]:
def determ(x: np.array, y: np.array) -> float:
    X_tmp = x.copy()
    
    ones  = np.ones((X_tmp.shape[0], 1))
    X_tmp = np.hstack([ones, X_tmp])
    
    b = np.linalg.inv(X_tmp.T.dot(X_tmp)).dot(X_tmp.T).dot(y)
    
    y_pred = X_tmp.dot(b)
    
    E = y - y_pred
    
    return 1 - E.var() / y.var()

In [50]:
import itertools

R = []
for i, j in itertools.combinations(range(X.shape[1]), 2):
    R.append((i, j, determ(X[:, [i, j]], Y)))

r = sorted(R, key=lambda r: r[2])[-1]
r

(0, 2, 0.7634246238793152)

Получили, что признаки с индексами $0$ и $2$ дают наилучший ответ.

##### Пункт 2:
Проверим значимость этой зависимости с помощью распределение Фишера:

$$ F = \frac {\frac {R^2}{m}}{\frac {1-R^2}{n - m - 1}} $$

Где:

$ m = 2$, 

$k_1 = m$, 

$k_2 = n - m - 1$


In [51]:
m = 2
k_1 = 2
k_2 = X.shape[0] - m - 1

r = r[-1]
F = (r / k_1) / ((1 - r) / k_2)

F

11.29443912292265

Найдем критическую область при $\alpha = 0.05$

In [52]:
from scipy import stats

alpha = 0.05

t = stats.f.ppf(1 - alpha, k_1, k_2)
t

4.73741412777588

Критическая область: 
$$\Omega_\alpha = (4.7374, +\infty)$$

Проверим, попадает ли значение статистики в критическую область:

In [53]:
F > t

True

Попадает, значит уравнение регрессии признается статистически значимым.

### Задача 2
#### Условие: 
Для проведения `A/B-тестирования` сайта интернет-магазина были получены следующие данные: страница `A` была посещена `2509 раз`, из них `77` закончились совершением покупки, страница `B` была посещена `1465 раз`, `60` из них закончились совершением покупки. Является ли значимым отличие конверсии на страницах `A` и `B`?

##### Решение:

В данном случае можно применить однофакторный дисперсионный анализ.

Для каждой страницы построим выборку совершения покупки:

1 - была покупка

0 - покупки не было
